# **Data Preparation**

## Imports

In [68]:
import pandas as pd
import numpy as np
from pathlib import Path
import pandas as pd
import os

# **Tratar Valores Ausentes**

## Reparticao da Producao de Energia

In [69]:
df_producao = pd.read_csv("../data/raw/ReparticaoProducao.csv", na_values=-990, sep=';',skiprows=2)

# Resumo dos valores ausentes
missing_data_producao = pd.DataFrame({
    'Total Ausentes': df_producao.isnull().sum(),
    'Percentual (%)': (df_producao.isnull().mean() * 100).round(2)
})

print("\nResumo das colunas com valores ausentes:")
print(missing_data_producao)


Resumo das colunas com valores ausentes:
                               Total Ausentes  Percentual (%)
Data e Hora                                 0             0.0
Hídrica                                     0             0.0
Eólica                                      0             0.0
Solar                                       0             0.0
Biomassa                                    0             0.0
Ondas                                       0             0.0
Gás Natural - Ciclo Combinado               0             0.0
Gás natural - Cogeração                     0             0.0
Carvão                                      0             0.0
Outra Térmica                               0             0.0
Importação                                  0             0.0
Exportação                                  0             0.0
Bombagem                                    0             0.0
Injeção de Baterias                         0             0.0
Consumo Baterias            

## Intensidade Media do Vento a 10m

In [70]:
# Importando o arquivo CSV, tratando -990 como ausente
df_intesidadeVento10 = pd.read_csv("../data/raw/IntensidadeMediaVento10m.csv", na_values=-990, sep=';', decimal='.')

# Resumo dos valores ausentes
missing_data = pd.DataFrame({
    'Total Ausentes': df_intesidadeVento10.isnull().sum(),
    'Percentual (%)': (df_intesidadeVento10.isnull().mean() * 100).round(2)
})

print("\nResumo das colunas com valores ausentes:")
print(missing_data)



Resumo das colunas com valores ausentes:
         Total Ausentes  Percentual (%)
ANO                   0            0.00
MES                   0            0.00
DIA                   0            0.00
1200551            1981           49.32
1210622            1525           37.96
1200567              14            0.35
1200575              57            1.42
1200545             283            7.05
1210702             663           16.50
1200560              83            2.07
1210683             190            4.73
1200548              57            1.42
1200570               1            0.02
1210718             191            4.75
1210734              52            1.29
1200571             226            5.63
1200579              32            0.80
1210770             192            4.78
1200558              28            0.70
1200562             471           11.73
1200554             352            8.76


## Direcao Media do Vento a 10m

In [71]:
df_direcaoVento10 = pd.read_csv("../data/raw/DirecaoMediaVento10m.csv", na_values=-990, sep=';', decimal='.')

# Resumo dos valores ausentes
missing_data = pd.DataFrame({
    'Total Ausentes': df_direcaoVento10.isnull().sum(),
    'Percentual (%)': (df_direcaoVento10.isnull().mean() * 100).round(2)
})

print("\nResumo das colunas com valores ausentes:")
print(missing_data)


Resumo das colunas com valores ausentes:
         Total Ausentes  Percentual (%)
ANO                   0            0.00
MES                   0            0.00
DIA                   0            0.00
1200551            2540           63.23
1210622            1090           27.13
1200567              36            0.90
1200575              47            1.17
1200545             617           15.36
1210702             665           16.55
1200560              85            2.12
1210683             191            4.75
1200548              63            1.57
1200570               1            0.02
1210718             207            5.15
1210734              62            1.54
1200571            1052           26.19
1200579              36            0.90
1210770             192            4.78
1200558              36            0.90
1200562             287            7.14
1200554            1026           25.54


## Temperatura Media

In [72]:
df_temperaturamedia = pd.read_csv("../data/raw/TemperaturaMedia.csv", na_values=-990, sep=';', decimal='.')

# Resumo dos valores ausentes
missing_data = pd.DataFrame({
    'Total Ausentes': df_temperaturamedia.isnull().sum(),
    'Percentual (%)': (df_temperaturamedia.isnull().mean() * 100).round(2)
})

print("\nResumo das colunas com valores ausentes:")
print(missing_data)


Resumo das colunas com valores ausentes:
         Total Ausentes  Percentual (%)
ANO                   0            0.00
MES                   0            0.00
DIA                   0            0.00
1200551             288            7.17
1210622             240            5.97
1200567              14            0.35
1200575              49            1.22
1200545             512           12.75
1210702              98            2.44
1200560              32            0.80
1210683             198            4.93
1200548              94            2.34
1200570               5            0.12
1210718             343            8.54
1210734              48            1.19
1200571             332            8.26
1200579              51            1.27
1210770             297            7.39
1200558              30            0.75
1200562             594           14.79
1200554              31            0.77


## **Estrategias para lidar com os valores ausentes**

### Para a Intensidade media do vento optamos por quando a percentagem de valores ausentes for superior a 30% em cada estação, usamos interpolação linear; nas restantes utilizámos a mediana

In [73]:
missing_percent = (df_intesidadeVento10.isnull().mean() * 100)


df_intesidadeVento10_treated = df_intesidadeVento10.copy()

for col in df_intesidadeVento10.columns:
    if missing_percent[col] > 30:
        # Se mais de 30% de ausentes → interpolar
        df_intesidadeVento10_treated[col] = df_intesidadeVento10_treated[col].interpolate(method='linear', limit_direction='both')
        # Preencher o que ainda sobrou (bordas) com a mediana
        mediana = df_intesidadeVento10_treated[col].median()
        df_intesidadeVento10_treated[col] = df_intesidadeVento10_treated[col].fillna(mediana)
    elif missing_percent[col] < 30:
        # Se menos de 5% de ausentes → preencher com mediana
        mediana = df_intesidadeVento10_treated[col].median()
        df_intesidadeVento10_treated[col] = df_intesidadeVento10_treated[col].fillna(mediana)
    else:
        pass

# Resultado final
print("\nResumo dos valores ausentes após tratamento:")
print(df_intesidadeVento10_treated.isnull().sum())


Resumo dos valores ausentes após tratamento:
ANO        0
MES        0
DIA        0
1200551    0
1210622    0
1200567    0
1200575    0
1200545    0
1210702    0
1200560    0
1210683    0
1200548    0
1200570    0
1210718    0
1210734    0
1200571    0
1200579    0
1210770    0
1200558    0
1200562    0
1200554    0
dtype: int64


In [74]:
os.makedirs('../data/processed', exist_ok=True)

# Salvando o dataset tratado em CSV
df_intesidadeVento10_treated.to_csv('../data/processed/IntensidadeMediaVento10m_processed.csv', index=False)


### Para a temperatura media subtituimos os valores usentes pela media.

In [75]:
df_ttmed = df_temperaturamedia.copy()

In [76]:
media_global = df_ttmed.select_dtypes(include=['float64', 'int64']).stack().mean()

df_ttmed.fillna(media_global, inplace=True)

df_ttmed.to_csv('../data/processed/TemperaturaMedia_processed.csv', index=False)

df_ttmed.isnull().sum()


ANO        0
MES        0
DIA        0
1200551    0
1210622    0
1200567    0
1200575    0
1200545    0
1210702    0
1200560    0
1210683    0
1200548    0
1200570    0
1210718    0
1210734    0
1200571    0
1200579    0
1210770    0
1200558    0
1200562    0
1200554    0
dtype: int64

### Para a direcao media do vento utilizamos a mediana circular


In [77]:
# Função para calcular a mediana circular (em graus)
def circular_median(values):
    # Converte os valores para radianos
    radians = np.deg2rad(values)

    # Calcula o seno e cosseno da média
    median_radians = np.arctan2(np.mean(np.sin(radians)), np.mean(np.cos(radians)))
    
    # Converte de volta para graus
    return np.rad2deg(median_radians) % 360

In [78]:
df_ddmed = df_direcaoVento10.copy()

mediana_circular = circular_median(df_ddmed.select_dtypes(include=['float64', 'int64']).stack())

df_ddmed.fillna(mediana_circular, inplace=True)

print("\nResumo dos valores ausentes após o tratamento na folha DD_MED:")
print(df_ddmed.isnull().sum())

df_ddmed.to_csv('../data/processed/DirecaoMediaVento10m_processed.csv', index=False)



Resumo dos valores ausentes após o tratamento na folha DD_MED:
ANO        0
MES        0
DIA        0
1200551    0
1210622    0
1200567    0
1200575    0
1200545    0
1210702    0
1200560    0
1210683    0
1200548    0
1200570    0
1210718    0
1210734    0
1200571    0
1200579    0
1210770    0
1200558    0
1200562    0
1200554    0
dtype: int64


## Agregação dos Dados

### Começar por extrair só o que queremos do Dataset da Repartição da Produção

In [79]:
df_producao_processed = df_producao.copy()

df_producao_processed.columns = df_producao_processed.columns.str.strip()
df_producao_processed = df_producao_processed[['Data e Hora', 'Eólica']]

# Ensure 'Data e Hora' is in datetime format
df_producao_processed['Data e Hora'] = pd.to_datetime(df_producao_processed['Data e Hora'])

df_producao_processed.set_index('Data e Hora', inplace=True)
df_diario = df_producao_processed.resample('D').sum().reset_index()
df_diario.rename(columns={'Data e Hora': 'Data'}, inplace=True)

### Para os datasets do vento temos que mudar o formato da data

In [81]:
df_ddmed_processed = df_ddmed.copy()
df_ddmed_processed = df_ddmed_processed.rename(columns={'ANO': 'year', 'MES': 'month', 'DIA': 'day'})
df_intesidadeVento10_treated = df_intesidadeVento10_treated.rename(columns={'ANO': 'year', 'MES': 'month', 'DIA': 'day'})
df_ddmed_processed['Data'] = pd.to_datetime(df_ddmed_processed[['year', 'month', 'day']])
df_intesidadeVento10_treated['Data'] = pd.to_datetime(df_intesidadeVento10_treated[['year', 'month', 'day']])

# Estações são colunas com nomes numéricos
estacoes = [col for col in df_ddmed_processed.columns if col.isdigit()]

dd_rad = np.deg2rad(df_ddmed_processed[estacoes])
x = df_intesidadeVento10_treated[estacoes] * np.cos(dd_rad)
y = df_intesidadeVento10_treated[estacoes] * np.sin(dd_rad)

x_medio = x.mean(axis=1)
y_medio = y.mean(axis=1)

intensidade_media = np.sqrt(x_medio**2 + y_medio**2)
direcao_media = (np.rad2deg(np.arctan2(y_medio, x_medio))) % 360

df_nacional = pd.DataFrame({
    'Data': df_ddmed_processed['Data'],
    'Intensidade_Media': intensidade_media,
    'Direcao_Media': direcao_media
})

### para a temperatura também

In [83]:
df_ttmed_processed = df_ttmed.copy()

# Ensure the renaming persists by using inplace=True
df_ttmed_processed.rename(columns={'ANO': 'year', 'MES': 'month', 'DIA': 'day'}, inplace=True)
df_ttmed_processed['Data'] = pd.to_datetime(df_ttmed_processed[['year', 'month', 'day']])

estacoes = [col for col in df_ttmed_processed.columns if col.isdigit()]
temp_media = df_ttmed_processed[estacoes].mean(axis=1)

df_temp = pd.DataFrame({
    'Data': df_ttmed_processed['Data'],
    'Temperatura_Media': temp_media
})

In [85]:
# Juntar os datasets usando a coluna 'Data' como chave
df_merged = df_nacional.merge(df_temp, on='Data', how='inner')
df_merged = df_merged.merge(df_diario, on='Data', how='inner')

# Salvar o dataset combinado em um arquivo CSV
df_merged.to_csv('../data/processed/dados_combinados.csv', index=False)